# Preprocessing

In [46]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os, glob

path_dir = ' '
file_list = os.listdir(path_dir)
data_count = 0

for name in file_list:
    img = cv2.imread(path_dir+name)
    data_count+=1
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ret,img_result1 = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img_result1[985:,1262:]=0
    ##img_result2 = cv2.adaptiveThreshold(img_result1, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,7,30)
    ret, img_result2 = cv2.threshold(img_result1, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    ##kernel = np.ones((2, 2), np.uint8)
    ##result = cv2.dilate(img_result2, kernel, iterations = 1)
    kernel = np.ones((1,1),np.uint8)
    opening = cv2.morphologyEx(img_result2, cv2.MORPH_OPEN,kernel, iterations = 2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
       
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    result_dist_transform = cv2.normalize(dist_transform, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1)
    ret, sure_fg = cv2.threshold(dist_transform, 0.03*dist_transform.max(),255, cv2.THRESH_BINARY)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

     # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    #markers = cv2.watershed(img, markers)

    # Implement watershed algorithm
    markers1 = cv2.watershed(img, markers)
    img[markers1 == -1] = [255, 0, 0]
#----------------------------------------------------------------------------------------

    final = img.copy()
    #crop_image = img_result1.copy() #img_result1--> crop image=black background
    crop_image = gray.copy() #gray-->crop image=gray
    markers2 = markers1.astype(np.uint8)
    ret, result = cv2.threshold(markers2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    result[985:,1262:]=0
    #contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    contours, hierarchy = cv2.findContours(result, 
                                           cv2.RETR_LIST,
                                           cv2.CHAIN_APPROX_NONE)

    COLOR = (0, 0, 200)

    idx =1

    for cnt in contours :
        if cv2.contourArea(cnt) > 500 :
            x,y,width, height = cv2.boundingRect(cnt)
            cv2.rectangle(final, (x,y), (x+width, y+height), COLOR, 1)
    
            crop = crop_image[y:y+height, x:x+width]
            #cv2.imshow(f'_crop_{idx}',crop)
            cv2.imwrite(f'save_train/{data_count}_crop_{idx}.jpg', crop) #파일저장
            idx+=1
    cv2.imwrite(f'bounding_box_{data_count}.jpg',final)        
#-----------------------------------check-------------------------------------
#final = copy한 원본이미지에 marker적용->bounding box측정
#crop_image = bounding box정보를 gray.copy()에 적용해서 crop --> bounding box없는 crop image로 저장가능
#cv2.imshow("result", result)
#cv2.imshow('img',img)
#cv2.imshow('final',final)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

##### crop image reference

In [ ]:
import cv2
img = cv2.imread('hela.jpg')
target_img = img.copy() #copy image

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours, hierarchy = cv2.findContours(otsu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

COLOR = (0, 200, 0)

idx =1
for cnt in contours :
    if cv2.contourArea(cnt) > 80 :
        x,y,width, height = cv2.boundingRect(cnt)
        cv2.rectangle(target_img, (x,y), (x+width, y+height), COLOR, 2)
    
        crop = img[y:y+height, x:x+width]
        #cv2.imshow(f'_crop_{idx}',crop)
        cv2.imwrite(f'_crop_{idx}.jpg', crop) #save file
        idx+=1
    
cv2.imshow('img', img)
cv2.imshow('gray', gray)
cv2.imshow('otsu', otsu)
cv2.imshow('contour', target_img)

cv2.waitKey(0)
cv2.destroyAllWindows()